<a id='item4'></a>

# Applied Data Science Capstone Project- The Battle of Neighborhoods

# Project Title: Exploring Montreal’s Food Diversity

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Introduction and Business Description </a>

2. <a href="#item2">Data description and sources of data</a>

3. <a href="#item3">Methodology and Analysis Output</a>

    3.1 <a href="item3.1"> Map of Montreal Neighbourhood</a>

    3.2 <a href="#item3.2">Neighborhood and Venue Association</a>
    
    3.3 <a href="#item3.3">Top 5 frequent venues and 10 most common venues in each neighborhood
    
    3.4 <a href="#item3.4"> Clustering of the Neighborhoods</a>
    
    3.5 <a href="#item3.5">Common venues in each neighborhood of the individual clusters</a>    
4. <a href="#item4">Results</a>    
5. <a href="#item3">Discussion</a>   
6. <a href="#item3">Recommendations</a>   
7. <a href="#item3">Conclusion</a>
</font>
</div>

##  1. Introduction and Business Description (Week -1)

##### Problem Background and Importance 

I currently reside in Montreal and am positively charged by its vibrant environment and surroundings. Montreal is metropolitan city in the province of Quebec on the Island of Montreal in Canada. Its population in 2019 was 4,195,523. It has been projected to grow to 4,377,306 by 2025 and 4,572,740 by 2030 ( http://worldpopulationreview.com/world-cities/montreal-population/). The main city has a population density of 4,517 people per square kilometer. Montreal has diverse ethnic groups including French, Italian, Irish, English, Scottish, and Spanish. Its visible minorities are black, Arab, Latin Americans, South Asians and Chinese.

To narrate multitude opportunities and scope of its evolving life style, I refer to a report (https://toolkit.mtl.org/en/studies-reports-and-statistics) published by Montreal Tourism. It is truly number one city, one may agree, from various perspectives including:

•	1st in North America and 20th in the world for hosting international events; 1st in North America and 10th in the world for reputation, 11th in the world for food diversity with 58 different types of cuisine

•	1st among smart cities in Canada and 2nd in North America, 2nd in the world for millennials, 1st university city in North America and 4th in the world

•	1st in Canada for the number of the world’s most sought-after hotels, 1st in the Americas and 15th in the world for hosting international meetings

•	Canada’s top city for hosting major sporting events,

•	Home to one of the North America’s top 10 airports

•	Home to one of the world’s coolest neighborhoods: little Italy.

•	2nd among North America’s top 15 cities for quality of life.

•	Famous Jazz festival attracts enormous number of tourists from across the globe during July - August.

•	More than one in every ten Canadians lives in the Montreal Metropolitan Area. https://canadapopulation.org/montreal-population/#Montreal_Metropopulation


Quebec culture is known for fine quality of dining with a style and the society is vibrant. Investment must be kept maintained or enhanced to keep up with above distinction of Montreal. Such ventures will support the peace and prosperities of Montreal, Quebec and Canada. These above factors, among others, will encourage the new entrepreneurs to open investments in the restaurants of generality or specialty, and for which they will need the scoping study, with some aspects provided in this project.

##### Problem Description:
The purpose of this project to provide a descriptive picture of food diversity in Montreal and its neighborhoods, and understand the factors, which can explain the diversity, along with any possible dependencies among these factors

#####  Target Audience
This project is targeted to the entrepreneurs who could be a start-up for opening a normal restaurant, a restaurant with specialty, or operation of nodes of its supply chain, of course at the place of significance in terms of low competition and operating cost, as well as to the clients [audience or stakeholders] of these selective places. as well as at the profitable related 

##### Success Criteria 
The identified venues with restaurants representing diverse foods and clusters of venues associated with specialty foods. 

slightly updated to synchronize with this report

https://www.tripsavvy.com/montreal-restaurants-4139153 \
https://www.tripsavvy.com/montreal-4139156    

## 2. Data description and sources of data (Week 1)

##### Data description and approach of data harvesting:

While moving on a number of paths to look for data, I came across link:
https://www.geonames.org/postal-codes/CA/QC/quebec.html
which contains the information on the first segment of the Canadian postal code, place (i.e. the neighborhood), its longitude and latitude, and Admin2 (Borough) it belongs to. 

I scraped this html file using BeautifulSoul and obtained the tabular data. 

Data from above link were wrangled in terms of seeing the spatial location of various boroughs and neighborhoods and filtering out some neighborhoods/boroughs which were seen too far away from the downtown Montreal on the map.

Thus the basic spatial data was cleaned and saved in a dataframe "neighborhoods" and was subjected to the analysis described in Section 3: Methodology.

#### The Codes for Data Harvested

In [4]:
from IPython.display import IFrame
IFrame(src="https://www.geonames.org/postal-codes/CA/QC/quebec.html", width='100%', height='200px')

#### Import libraries and install Beautifulsoup4

In [5]:
import pandas as pd
import sys
sys.path.append('C:\\Python27\\Scripts\\pip.exe')
!pip install beautifulsoup4

!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup
import requests
import urllib.request
from lxml import html

     |████████████████████████████████| 112kB 29.6MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 8.1MB/s eta 0:00:01


##### Scrape the web page containing Canadian postal codes, boroughs and neighborhoods

In [6]:
# Scrape the page with BeautifulSoup
quote_page='https://www.geonames.org/postal-codes/CA/QC/quebec.html'
page = requests.get(quote_page)
#print(page.status_code)

soup= BeautifulSoup(page.text, "html.parser")
right_table=soup.find('table', {"class" : "restable"})
#print(right_table)

# Transfer to a DataFrame 
df0 = pd.read_html(str(right_table))[0]

print(df0.shape) 
df0.head()

(401, 7)


,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
1,NaN,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192
2,2.0,Duvernay-Est,H7A,Canada,Quebec,NaN,NaN
3,NaN,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592
4,3.0,Saint-Vincent-de-Paul,H7C,Canada,Quebec,NaN,NaN


#### Create a dataframe of Borough, Postal code, Neighborhood, Latitude and Longitude of the Neighborhood
#### Here: Place = Neighborhood, Code = Postal code, Admin2 = Borough, Admin2=NaN removed

In [7]:
# 
col_names=['Place', 'Code','Admin2', 'Admin3']
df1=pd.DataFrame(columns=col_names)

import numpy as np
odd=list(range(0,401,2))
even=list(range(1,400,2))

df1 = df0.drop(even)
df1
df1=df1.drop(200)
df2=df0.drop(odd)
df2=df2.drop(df2.columns[[0,2,3,4,5,6]], axis=1)
df2[['Latitude','Longitude']]=df2.Place.str.split('/',expand=True)
df2=df2.drop(columns=['Place'])
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df1
df2
df3=pd.concat([df1,df2], axis=1)
df3['Latitude'] =df3['Latitude'].astype(float)
df3['Longitude'] = df3['Longitude'].astype(float)
df3=df3.drop(columns=['Country','Admin1','Admin3'])
df3.rename(columns={'Place':'Neighborhood','Admin2':'Borough'}, inplace=True)
ListCodes=['H2Z','H3A','H3B','H3G','H3H','H3Y','H3Z',   'H1T', 'H1X', 'H1Y', 
           'G1G','G2N',   'H2T','H2W','H2X','H3P','H3R',  'H3S', 'H3T','H3V','H3W',  'H4A','H4B', 
           'H4R','H4S','H4T']

#Removed outer neighborhoods etc: Dorval outer H9P ... St laura outer H4N, cote-St-luc west H4W 
# more removed.. H1H, H1B, H1G, H4X, H4L, H4M, G2M(Jean-Talon Northeast)
neighborhoods= df3[df3['Code'].isin(ListCodes)].reset_index(drop=True)
neighborhoods=neighborhoods.drop(neighborhoods.index[0])
neighborhoods=neighborhoods.reset_index(drop=True)
neighborhoods=neighborhoods[['Borough','Neighborhood',  'Latitude', 'Longitude', 'Code']]

# The English and French spelling for Montreal should be one. Also Pleateu Mont-Royal as Mount Royal
neighborhoods['Borough']=neighborhoods['Borough'].str.replace("Montréal", "Montreal", case=False)
neighborhoods['Borough']=neighborhoods['Borough'].str.replace("Plateau Mont-Royal", "Mount Royal", case=False)

neighborhoods=neighborhoods.sort_values('Borough')
neighborhoods=neighborhoods.reset_index(drop=True)
print(neighborhoods.shape)
neighborhoods.head()

(25, 5)


,Borough,Neighborhood,Latitude,Longitude,Code
0,Côte-des-Neiges,Côte-des-Neiges Southwest,45.499,-73.609,H3W
1,Côte-des-Neiges,Côte-des-Neiges East,45.502,-73.619,H3V
2,Côte-des-Neiges,Côte-des-Neiges Northeast,45.506,-73.630,H3T
3,Côte-des-Neiges,Côte-des-Neiges North,45.510,-73.648,H3S
4,Jean-Talon,Jean-Talon West,46.914,-71.340,G2N


## 3 Methodology and Analysis Output

Mapping of the neighborhood; Use of Foursquare to retrieve the venues information; testing independence of venue and neighborhood (assumptions ignored) using contingency chi-square test; Neighborhood x venue specificity: frequency of top 5 venues and up to 10 most common venues neighborhood-wise; cluster of the neighborhoods;up to 10 most common venues neighborhood-wise in each cluster.

Examine the data for each neighborhood.

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude,Code
0,Côte-des-Neiges,Côte-des-Neiges Southwest,45.499,-73.609,H3W
1,Côte-des-Neiges,Côte-des-Neiges East,45.502,-73.619,H3V
2,Côte-des-Neiges,Côte-des-Neiges Northeast,45.506,-73.630,H3T
3,Côte-des-Neiges,Côte-des-Neiges North,45.510,-73.648,H3S
4,Jean-Talon,Jean-Talon West,46.914,-71.340,G2N
5,Montreal,Westmount East,45.488,-73.605,H3Z
6,Montreal,Westmount West,45.490,-73.631,H3Y
7,Montreal,Downtown Montreal Northeast,45.512,-73.568,H2Z
8,Montreal,Downtown Montreal North,45.505,-73.562,H3A
9,Montreal,Downtown Montreal East,45.504,-73.575,H3B


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),\
                                len(neighborhoods['Neighborhood'].unique()) )  )

The dataframe has 7 boroughs and 25 neighborhoods.


#### Map of Montreal neighborhoods

<a id='item1'></a>

In [10]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if required
import folium # map rendering library
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


In [11]:
!conda install -c conda-forge geopy --yes # uncomment this line if required
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 3

<a id='item1'></a>

#### Use geopy library to get the latitude and longitude values of Montreal.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>montreal_explorer</em>, as shown below.

In [13]:
address = 'Montreal, QC, CA'

geolocator = Nominatim(user_agent="montreal_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

GeocoderTimedOut: Service timed out

#### Create a map of Montreal with circles superimposed on neighborhoods.

In [2]:
# create map of Montreal using latitude and longitude values
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

NameError: name 'folium' is not defined

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'WZWCI02QWH40BHLHT4PZ34AO5P2KHYE04P1FY0BAVBBBAF52' # your Foursquare ID
CLIENT_SECRET = 'LWAKNLJK0DKJVM2J0UZ4V50FYEK30FAW2I1QRGMTVBJ43WHS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZWCI02QWH40BHLHT4PZ34AO5P2KHYE04P1FY0BAVBBBAF52
CLIENT_SECRET:LWAKNLJK0DKJVM2J0UZ4V50FYEK30FAW2I1QRGMTVBJ43WHS


#### Let's explore the 11-th neighborhood in our dataframe. This is in the middle of downtown.

Get the neighborhood's name.

In [62]:
neighN =12
neighborhoods.loc[neighN-1,'Neighborhood']

'Downtown Montreal South & West'

Get the neighborhood's latitude and longitude values.

In [63]:
neighborhood_latitude = neighborhoods.loc[neighN-1,'Latitude']                # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[neighN-1,'Longitude']  # neighborhood longitude value

neighborhood_name = neighborhoods.loc[neighN-1,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown Montreal South & West are 45.499, -73.579.


##### Now, let's get the top 100 venues that are in Southwest Downtown Montreal South & West within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [64]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WZWCI02QWH40BHLHT4PZ34AO5P2KHYE04P1FY0BAVBBBAF52&client_secret=LWAKNLJK0DKJVM2J0UZ4V50FYEK30FAW2I1QRGMTVBJ43WHS&v=20180605&ll=45.499,-73.579&radius=500&limit=100'

Send the GET request and examine the resutls

In [65]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e17ec61618f43001b891324'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ville-Marie',
  'headerFullLocation': 'Ville-Marie, Montreal',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 144,
  'suggestedBounds': {'ne': {'lat': 45.503500004500005,
    'lng': -73.57259186283588},
   'sw': {'lat': 45.4944999955, 'lng': -73.58540813716411}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e356969b61cddd1cd3cbba3',
       'name': 'Musée des beaux-arts de Montréal (MBAM)',
       'location': {'address': '1380 rue Sherbrooke O',
        'crossStreet': 'coin du Musée',
        'lat': 45.498436,
        'lng': -73.579715,
        'labe

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [67]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Musée des beaux-arts de Montréal (MBAM),Art Museum,45.498436,-73.579715
1,The Ritz-Carlton Montréal,Hotel,45.500191,-73.578127
2,La Panthère Verte,Vegetarian / Vegan Restaurant,45.497335,-73.580144
3,Maison Boulud,French Restaurant,45.500062,-73.578324
4,Mandy's,Salad Place,45.498286,-73.577895


And how many venues were returned by Foursquare?

In [68]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Montreal

#### Let's create a function to repeat the same process to all the neighborhoods in Montreal

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Now write the code to run the above function on each neighborhood and create a new dataframe called *montreal_venues*.

In [70]:
# 
montreal_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Côte-des-Neiges Southwest
Côte-des-Neiges East
Côte-des-Neiges Northeast
Côte-des-Neiges North
Jean-Talon West
Westmount East
Westmount West
Downtown Montreal Northeast
Downtown Montreal North
Downtown Montreal East
Downtown Montreal Southeast
Downtown Montreal South & West
Mount Royal Central
Mount Royal North
Plateau Mont-Royal Southeast
Plateau Mont-Royal South Central
Plateau Mont-Royal West
Notre-Dame-de-GrÔce Northeast
Notre-Dame-de-GrÔce Southwest
Rosemont South
Rosemont Central
Rosemont North
Saint-Laurent Central
Saint-Laurent Southwest
Saint-Laurent Southeast


#### Let's check the size of the resulting dataframe

In [71]:
print(montreal_venues.shape)
montreal_venues
montreal_venues.head()

(683, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Côte-des-Neiges Southwest,45.499,-73.609,Pug You,45.497383,-73.606553,Business Service
1,Côte-des-Neiges Southwest,45.499,-73.609,Allia Design & Cultures,45.497506,-73.605296,Furniture / Home Store
2,Côte-des-Neiges Southwest,45.499,-73.609,Depanneur 2000,45.495465,-73.608230,Convenience Store
3,Côte-des-Neiges Southwest,45.499,-73.609,STM Arrêt/Stop #51487,45.495481,-73.607069,Bus Station
4,Côte-des-Neiges East,45.502,-73.619,Coop HEC Montréal,45.504099,-73.622041,College Bookstore


Let's check how many venues were returned for each neighborhood

In [72]:
montreal_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Côte-des-Neiges East,6,6,6,6,6,6
Côte-des-Neiges North,4,4,4,4,4,4
Côte-des-Neiges Northeast,5,5,5,5,5,5
Côte-des-Neiges Southwest,4,4,4,4,4,4
Downtown Montreal East,79,79,79,79,79,79
Downtown Montreal North,100,100,100,100,100,100
Downtown Montreal Northeast,82,82,82,82,82,82
Downtown Montreal South & West,100,100,100,100,100,100
Downtown Montreal Southeast,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [73]:
print('There are {} uniques categories.'.format(len(montreal_venues['Venue Category'].unique())))

There are 174 uniques categories.


### 3.2 Neighborhood and Venue Association using chisquare contingency tests (assumed independence of factor levels within)

In [74]:
from scipy.stats import chi2_contingency

RC = montreal_venues[['Venue Category','Neighborhood']]
RC=RC.rename(columns={'Venue Category': 'Venue_Category'})

RC=pd.crosstab(RC.Venue_Category, RC.Neighborhood)
print(RC.shape)
RC.to_numpy()
print(chi2_contingency(RC))
Venues=RC.sum(axis=1) *100 /RC.sum(axis=1).sum()
print(Venues.sort_values( ascending=False))

(174, 25)
(5460.871678624702, 1.8907772708294528e-39, 4152, array([[0.00878477, 0.00585652, 0.00732064, ..., 0.00146413, 0.00732064,
        0.01317716],
       [0.00878477, 0.00585652, 0.00732064, ..., 0.00146413, 0.00732064,
        0.01317716],
       [0.04392387, 0.02928258, 0.03660322, ..., 0.00732064, 0.03660322,
        0.0658858 ],
       ...,
       [0.00878477, 0.00585652, 0.00732064, ..., 0.00146413, 0.00732064,
        0.01317716],
       [0.00878477, 0.00585652, 0.00732064, ..., 0.00146413, 0.00732064,
        0.01317716],
       [0.04392387, 0.02928258, 0.03660322, ..., 0.00732064, 0.03660322,
        0.0658858 ]]))
Venue_Category
Café                   6.734993
Hotel                  3.953148
French Restaurant      2.928258
Bar                    2.928258
Coffee Shop            2.781845
                         ...   
Donut Shop             0.146413
Electronics Store      0.146413
Empanada Restaurant    0.146413
English Restaurant     0.146413
Accessories Store      0.14

#### Result: The type of business (venue) is not independent of the neighborhood. This indicates a point in examining neighborhood-wise.

#### Results: overall the nebhorhoods, the top ten venues are
- "Cafe', Hotel, French restaurant, Bar and Coffee Shop", Park, Restaurant, Pizza Place, Bakery, Clothing Store"
- Cafe' and Coffee Shop are within top ten; entrepreneur may look for general coffee places as well as specialty coffeee
- French Restaurant and Restaurant appear in top ten, shows the popularity of specialty restauarant.
- Cafe is number 1 and Hotel is 2 which is easy to understand considering the nature of Montreal.  

The venues with lower appearances are in these neighborhoods:
- Donut Shop, Electronics Store, Empanada Restaurant, English Restaurant, Accessories Store
- These reflect a messagae to investors on updating the facilities in these specialty.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [75]:
# one hot encoding
montreal_onehot = pd.get_dummies(montreal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['Neighborhood'] = montreal_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,Yoga Studio,Accessories Store,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [76]:
montreal_onehot.shape

(683, 174)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
montreal_grouped = montreal_onehot.groupby('Neighborhood').mean().reset_index()
montreal_grouped

,Neighborhood,Yoga Studio,Accessories Store,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Côte-des-Neiges East,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Côte-des-Neiges North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Côte-des-Neiges Northeast,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Côte-des-Neiges Southwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Downtown Montreal East,0.012658,0.000000,0.000000,0.000000,0.012658,0.012658,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.025316,0.000000,0.000000,0.00
5,Downtown Montreal North,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.00
6,Downtown Montreal Northeast,0.000000,0.000000,0.012195,0.024390,0.024390,0.000000,0.012195,0.000000,0.012195,...,0.012195,0.012195,0.000000,0.012195,0.000000,0.000000,0.024390,0.012195,0.012195,0.00
7,Downtown Montreal South & West,0.010000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.01
8,Downtown Montreal Southeast,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Jean-Talon West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


#### Let's confirm the new size

In [78]:
montreal_grouped.shape

(25, 174)

#### Neighborhood associated venues
#### Let's print each neighborhood along with the top 5 most common venues

In [79]:
num_top_venues = 5

for hood in montreal_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = montreal_grouped[montreal_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Côte-des-Neiges East----
               venue  freq
0       Burger Joint  0.17
1      Metro Station  0.17
2               Café  0.17
3  French Restaurant  0.17
4           Pharmacy  0.17


----Côte-des-Neiges North----
                   venue  freq
0            Flower Shop  0.25
1  Portuguese Restaurant  0.25
2       Sushi Restaurant  0.25
3                   Park  0.25
4            Pastry Shop  0.00


----Côte-des-Neiges Northeast----
               venue  freq
0  Food & Drink Shop   0.2
1               Park   0.2
2        Supermarket   0.2
3   Sushi Restaurant   0.2
4      Grocery Store   0.2


----Côte-des-Neiges Southwest----
                    venue  freq
0       Convenience Store  0.25
1  Furniture / Home Store  0.25
2        Business Service  0.25
3             Bus Station  0.25
4           Poutine Place  0.00


----Downtown Montreal East----
            venue  freq
0           Hotel  0.10
1  Clothing Store  0.06
2     Coffee Shop  0.06
3            Café  0.06
4  Sandwich 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = montreal_grouped['Neighborhood']

for ind in np.arange(montreal_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(montreal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted  #.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Côte-des-Neiges East,Burger Joint,Pharmacy,Café,College Bookstore,Metro Station,French Restaurant,Discount Store,English Restaurant,Empanada Restaurant,Electronics Store
1,Côte-des-Neiges North,Portuguese Restaurant,Sushi Restaurant,Flower Shop,Park,Coworking Space,Discount Store,Empanada Restaurant,Electronics Store,Convenience Store,Dumpling Restaurant
2,Côte-des-Neiges Northeast,Park,Food & Drink Shop,Supermarket,Sushi Restaurant,Grocery Store,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
3,Côte-des-Neiges Southwest,Convenience Store,Bus Station,Business Service,Furniture / Home Store,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
4,Downtown Montreal East,Hotel,Coffee Shop,Café,Clothing Store,Sandwich Place,Cosmetics Shop,Gym,Gastropub,Deli / Bodega,Men's Store
5,Downtown Montreal North,Hotel,French Restaurant,Café,Plaza,Cocktail Bar,Chinese Restaurant,Restaurant,Asian Restaurant,Sandwich Place,Italian Restaurant
6,Downtown Montreal Northeast,Café,Hotel,French Restaurant,Concert Hall,Bar,Pizza Place,Coffee Shop,Restaurant,Brewery,Breakfast Spot
7,Downtown Montreal South & West,Café,Art Museum,Hotel,Italian Restaurant,French Restaurant,Pub,Pizza Place,Middle Eastern Restaurant,Burger Joint,Clothing Store
8,Downtown Montreal Southeast,Convenience Store,Mediterranean Restaurant,Soccer Field,Park,Cupcake Shop,Cycle Studio,Empanada Restaurant,Cosmetics Shop,Electronics Store,Dumpling Restaurant
9,Jean-Talon West,Sandwich Place,Restaurant,Burger Joint,Dessert Shop,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


<a id='item4'></a>

## 3.4 Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [82]:
# set number of clusters
kclusters = 5   

montreal_grouped_clustering = montreal_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(montreal_grouped_clustering)

# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10].astype(int)

array([4, 2, 2, 1, 4, 4, 4, 4, 4, 4])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [83]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

montreal_merged = neighborhoods

# montreal_grouped with montreal venues to add latitude/longitude for each neighborhood
montreal_merged = montreal_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

montreal_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Côte-des-Neiges,Côte-des-Neiges Southwest,45.499,-73.609,H3W,1,Convenience Store,Bus Station,Business Service,Furniture / Home Store,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
1,Côte-des-Neiges,Côte-des-Neiges East,45.502,-73.619,H3V,4,Burger Joint,Pharmacy,Café,College Bookstore,Metro Station,French Restaurant,Discount Store,English Restaurant,Empanada Restaurant,Electronics Store
2,Côte-des-Neiges,Côte-des-Neiges Northeast,45.506,-73.630,H3T,2,Park,Food & Drink Shop,Supermarket,Sushi Restaurant,Grocery Store,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
3,Côte-des-Neiges,Côte-des-Neiges North,45.510,-73.648,H3S,2,Portuguese Restaurant,Sushi Restaurant,Flower Shop,Park,Coworking Space,Discount Store,Empanada Restaurant,Electronics Store,Convenience Store,Dumpling Restaurant
4,Jean-Talon,Jean-Talon West,46.914,-71.340,G2N,4,Sandwich Place,Restaurant,Burger Joint,Dessert Shop,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


Finally, let's visualize the resulting clusters

In [84]:
# create map
latitude=  45.5017
longitude= -73.5673   #for Montreal
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(montreal_merged['Latitude'], montreal_merged['Longitude'], montreal_merged['Neighborhood'], montreal_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. The cluster names are mainly assigned based on the neighborhoods and their geography. 

#### Cluster 1: Saint-Laurent Southwest

In [85]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 0, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Saint-Laurent Southwest,0,Park,Women's Store,Diner,English Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


#### Cluster 2: Côte-des-Neiges Southwest

In [86]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 1, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Côte-des-Neiges Southwest,1,Convenience Store,Bus Station,Business Service,Furniture / Home Store,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
19,Rosemont South,1,Portuguese Restaurant,Deli / Bodega,Bus Station,Business Service,Chinese Restaurant,Convenience Store,Pet Store,Cycle Studio,Dog Run,Empanada Restaurant
24,Saint-Laurent Southeast,1,Construction & Landscaping,Business Service,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run


#### Cluster 3: Côte-des-Neiges Northeast

In [87]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 2, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Côte-des-Neiges Northeast,2,Park,Food & Drink Shop,Supermarket,Sushi Restaurant,Grocery Store,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
3,Côte-des-Neiges North,2,Portuguese Restaurant,Sushi Restaurant,Flower Shop,Park,Coworking Space,Discount Store,Empanada Restaurant,Electronics Store,Convenience Store,Dumpling Restaurant
5,Westmount East,2,Park,Athletics & Sports,Tennis Court,Scenic Lookout,Women's Store,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
17,Notre-Dame-de-GrÔce Northeast,2,Grocery Store,Accessories Store,Pizza Place,Athletics & Sports,Diner,Sushi Restaurant,Park,Dim Sum Restaurant,Empanada Restaurant,Electronics Store


#### Cluster 4: Mount Royal Central

In [88]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 3, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Mount Royal Central,3,Basketball Court,Gym / Fitness Center,Women's Store,Diner,English Restaurant,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run


#### Cluster 5: Montreal Downtown

In [89]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 4, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Côte-des-Neiges East,4,Burger Joint,Pharmacy,Café,College Bookstore,Metro Station,French Restaurant,Discount Store,English Restaurant,Empanada Restaurant,Electronics Store
4,Jean-Talon West,4,Sandwich Place,Restaurant,Burger Joint,Dessert Shop,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store
6,Westmount West,4,Vietnamese Restaurant,Department Store,Coffee Shop,Filipino Restaurant,Sri Lankan Restaurant,Gym,Fast Food Restaurant,Art Gallery,Cycle Studio,Dog Run
7,Downtown Montreal Northeast,4,Café,Hotel,French Restaurant,Concert Hall,Bar,Pizza Place,Coffee Shop,Restaurant,Brewery,Breakfast Spot
8,Downtown Montreal North,4,Hotel,French Restaurant,Café,Plaza,Cocktail Bar,Chinese Restaurant,Restaurant,Asian Restaurant,Sandwich Place,Italian Restaurant
9,Downtown Montreal East,4,Hotel,Coffee Shop,Café,Clothing Store,Sandwich Place,Cosmetics Shop,Gym,Gastropub,Deli / Bodega,Men's Store
10,Downtown Montreal Southeast,4,Convenience Store,Mediterranean Restaurant,Soccer Field,Park,Cupcake Shop,Cycle Studio,Empanada Restaurant,Cosmetics Shop,Electronics Store,Dumpling Restaurant
11,Downtown Montreal South & West,4,Café,Art Museum,Hotel,Italian Restaurant,French Restaurant,Pub,Pizza Place,Middle Eastern Restaurant,Burger Joint,Clothing Store
13,Mount Royal North,4,Discount Store,Pharmacy,Shopping Mall,Asian Restaurant,Bookstore,Gym,Liquor Store,Women's Store,Empanada Restaurant,Electronics Store
14,Plateau Mont-Royal Southeast,4,Café,Portuguese Restaurant,Bakery,Bar,Japanese Restaurant,Breakfast Spot,French Restaurant,Burger Joint,Steakhouse,Ice Cream Shop


# 4. Results

 ##### The analyzed data set had a total of 25 neighborhoods from a total of 7 boroughs in Montreal, QC area. 
 ##### The 173 venues were returned by Foursquare when explored within a circular area of radius 500 m. 
 ##### The contingency table of Neighborhood X Venue showed significant departure from independence between them (P = 7.58 x E-40)
 ##### Such a high degree of association indicates business decision for selection of venues must be made specific to the neighborhood. This indicates the point in examining neighborhood-wise.
 ##### Based on a view from the bridge overall the neighborhoods, the top ten venues are:
 - "Cafe', Hotel, French restaurant, Bar and Coffee Shop", Park, Restaurant, Pizza Place, Bakery, Clothing Store"
 - Cafe' and Coffee Shop are within top ten; entrepreneur may look for general coffee places as well as specialty coffeee
 - French Restaurant and Restaurant appear in top ten, shows the popularity of specialty restauarant.
 - Cafe is number 1 and Hotel is 2 which is easy to understand considering the nature of Montreal.  

The venues with lower appearances are in these neighborhoods:
 - Soccer Field, Women's Store, Korean Restaurant, Laser Tag, Arepa Restaurant (lowest most).

 - These reflect a messagae to investors on updating the facilities in these specialty.

## 5 Discussion 

The position of Montreal is unique from various perspective of its welcoming nature to entrepreneurs to explore it rich French cultural heritage and ease of doing business in Montreal and Quebec. This study mainly focused on businesses in the areas around downtown Montreal and shorter extension around it. As we find there is tremendous diversity in the venues within and across the neighborhoods. These numbers and names in the Table 1 and other tables in the notebook clearly provide a picture to choose the popular or specialty venues and possibly do the scoping study to further consider the investment.

## 6 Recommendations

1.	The tables of the common venues of importance may be considered for exploring any investment in the area of interest to Montreal population and tourists.
2.	Investors may consider further enhancing the food diversity in the specialty restaurants such as French, Japanese (Shushi), Portuguese, English, Mediterranean, Spanish, Chinese, Indian, Sri Lankan, Vietnamese, Filipino cuisines among others. 
3.	Since majority of such activities are in largest cluster, the suitable neighborhood/location in this cluster may be considered.

The analysis details are available in the links to the Notebook and a summary in a presentation file on Github repository.

## 7 Conclusion

The assignment on the exploration of diversity of food and restaurants in wider Montreal area. The study undertook a number of tools and techniques including, scraping of web page using BeautifulSoup, pandas and other libraries in Python to harvest the data in dataframe and prepare for analysis. The Foursquare was used to extract the venues information in the neighborhoods of Montreal. It was found significant association between the neighborhood and venue. There was substantial diversity in the venues over the neighborhoods. It would be worth exploring effects on employment and economics contributed by these venues in the invidual neighborhoods and montreal region overall.

#### Report and Presentation files are in separate files with links via Github.

### Acknowlegdgements:

I thank Drs. [Alex Aklson](https://www.linkedin.com/in/aklson/)and [Polong Lin](https://www.linkedin.com/in/polonglin/) for creating this notebook which was duly inserted with parameters values in places in context of Montreal for this project. It would not have been in the form this project appears here.